In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import pandas as pd
from keras_preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa

In [2]:
directory = 'C:\datasets\img_align_celeba\img_align_celeba'

In [3]:
tag = pd.read_csv('C:/datasets/list_attr_celeba.csv')

In [4]:
tag = tag[['image_id','Male']]
tag.Male = np.where(tag.Male == 1, 1, 0)

In [5]:
train = tag.sample(frac = .8, random_state=42)
test = tag.drop(train.index)
train

,image_id,Male
93241,093242.jpg,1
121413,121414.jpg,0
10624,010625.jpg,1
82523,082524.jpg,0
73192,073193.jpg,0
...,...,...
36890,036891.jpg,1
1992,001993.jpg,0
47852,047853.jpg,0
79821,079822.jpg,0


In [6]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [7]:
train_generator=datagen.flow_from_dataframe(
dataframe=train,
directory = directory,
x_col="image_id",
y_col="Male",
subset="training",
batch_size=16,
seed=42,
shuffle=True,
    class_mode='raw',
target_size=(256,256))

valid_generator=datagen.flow_from_dataframe(
dataframe=train,
directory=directory,
x_col="image_id",
y_col="Male",
subset="validation",
batch_size=16,
seed=42,
shuffle=True,
class_mode="raw",
target_size=(256,256))

test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=test,
directory=directory,
x_col="image_id",
y_col='Male',
batch_size=16,
seed=42,
shuffle=False,
class_mode='raw',
target_size=(256,256))

Found 121560 validated image filenames.
Found 40519 validated image filenames.
Found 40520 validated image filenames.


In [11]:
class MLP(tf.keras.layers.Layer):
    def __init__(self, row_size, kernel_size):
        super(MLP, self).__init__()
        self.row_size = row_size
        self.kernel_size = kernel_size

        self.conv1 = tf.keras.layers.Conv1D(self.kernel_size, 1, input_shape = (self.row_size, self.kernel_size))
        self.conv2 = tf.keras.layers.Conv1D(self.kernel_size, 1)
        
    def call(self, X):
        X = self.conv1(X)
        X = tf.nn.gelu(X)
        X = self.conv2(X)
        return X

class Mixer_block(tf.keras.layers.Layer):
    def __init__(self, n_patches, hidden_chaneel_size):
        super(Mixer_block, self).__init__()
        self.n_patches = n_patches
        self.hidden_chaneel_size = hidden_chaneel_size
        
        self.ln = tf.keras.layers.LayerNormalization(axis = 1)
        self.t = tf.keras.layers.Permute((2,1))
        self.mlp1 = MLP(self.hidden_chaneel_size, self.n_patches)
        self.mlp2 = MLP(self.n_patches, self.hidden_chaneel_size)
        
    def call(self, X):
        skip_X = X
        X = self.ln(X)
        X = self.t(X)
        X = self.mlp1(X)
        X = self.t(X)
        X = X  + skip_X
        skip_X2 = X
        X = self.ln(X)
        X = self.mlp2(X)
        X = X + skip_X2
        return X

class MLP_Mixer(tf.keras.models.Model):
    def __init__(self, patch_size:int, image_size:int, hidden_channel_size:int, n_mixers:int, fc_nodes:int, fc_layers:int):
        tf.random.set_seed(42)
        super(MLP_Mixer, self).__init__()
        self.patch_size = patch_size
        self.image_size = image_size
        if (image_size % patch_size) != 0:
            raise ValueError('size error')
        self.n_patches = int((tf.square(self.image_size) / tf.square(self.patch_size)).numpy())
        self.hidden_channel_size = hidden_channel_size
        self.n_mixers = n_mixers
        self.fc_nodes = fc_nodes
        self.fc_layers = fc_layers
        
        self.patchConv = tf.keras.layers.Conv2D(self.hidden_channel_size, (self.patch_size, self.patch_size), strides = (self.patch_size, self.patch_size))
        self.reshapeL = tf.keras.layers.Reshape((self.n_patches, self.hidden_channel_size))
        self.mixer_blocks = [Mixer_block(self.n_patches, self.hidden_channel_size) for _ in range(self.n_mixers)]
        self.gap = tf.keras.layers.GlobalAveragePooling1D()
        self.fc = [tf.keras.layers.Dense(self.fc_nodes, activation = tfa.activations.gelu, kernel_initializer = tf.keras.initializers.lecun_normal(seed = i)) for i in range(self.fc_layers)]
        self.classifier = tf.keras.layers.Dense(1, activation = 'sigmoid')
        
    def call(self, X):
        X = self.patchConv(X)
        X = self.reshapeL(X)
        for mixer in self.mixer_blocks:
            X = mixer(X)
        X = self.gap(X)
        for fc_layer in self.fc:
            X = fc_layer(X)
        X = self.classifier(X)
        return X
    

In [13]:
mlp_mixer = MLP_Mixer(64, 256, 512, 5, 512, 2)
opt = tf.keras.optimizers.Adam(.0002)
mlp_mixer.compile(optimizer=opt, loss=tf.losses.binary_crossentropy, metrics='acc')

In [14]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
mlp_mixer.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10)

Epoch 1/10
7597/7597 [==============================] - 297s 39ms/step - loss: 0.2377 - acc: 0.8997 - val_loss: 0.1714 - val_acc: 0.9304
Epoch 2/10
7597/7597 [==============================] - 294s 39ms/step - loss: 0.1580 - acc: 0.9372 - val_loss: 0.1470 - val_acc: 0.9458
Epoch 3/10
7597/7597 [==============================] - 294s 39ms/step - loss: 0.1359 - acc: 0.9467 - val_loss: 0.1435 - val_acc: 0.9421
Epoch 4/10
7597/7597 [==============================] - 295s 39ms/step - loss: 0.1184 - acc: 0.9538 - val_loss: 0.1301 - val_acc: 0.9480
Epoch 5/10
7597/7597 [==============================] - 297s 39ms/step - loss: 0.1071 - acc: 0.9584 - val_loss: 0.1315 - val_acc: 0.9473
Epoch 6/10
7597/7597 [==============================] - 298s 39ms/step - loss: 0.0980 - acc: 0.9626 - val_loss: 0.1202 - val_acc: 0.9531
Epoch 7/10
7597/7597 [==============================] - 300s 39ms/step - loss: 0.0885 - acc: 0.9658 - val_loss: 0.1213 - val_acc: 0.9525
Epoch 8/10
7597/7597 [===================